# Uniswap V3 price analysis

In this notebook we will show how to download price events from Uniswap V3 to your computer as CSV files and use them to analyse price in each pool.

You will need to havev Ethereum JSON-RPC API node URL in order to pull out the data from Ethereum blockchain.
You will be interactively asked for this API key.

### Setup the notebook

In [311]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of eth_defi.uniswap_v3.pool failed: Traceback (most recent call last):
  File "/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-0IMGzMxl-py3.9/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-0IMGzMxl-py3.9/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-0IMGzMxl-py3.9/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-0IMGzMxl-py3.9/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-0IMGzMxl-py3.9/li

In [312]:
import pandas as pd
import matplotlib.pyplot as plt

from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv

### Download the raw data

For simplicity, you can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/), however we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node).

In [313]:
from web3 import Web3, HTTPProvider

# Get your node JSON-RPC URL
json_rpc_url = input("Please enter your Ethereum mainnet JSON-RPC URL here")
web3 = Web3(HTTPProvider(json_rpc_url))

As an example, here we download raw events from first few blocks after Uniswap V3 was deployed.
The events will be stored in several CSV files at `/tmp` folder.

The scan will take hours. It can resumed on a crash, as we save the last scanned block in a JSON state file.

In [314]:
from eth_defi.event_reader.json_state import JSONFileScanState

start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 250_000

# Stores the last block number of event data we store
state = JSONFileScanState("/tmp/uniswap-v3-price-scan.json")

fetch_events_to_csv(json_rpc_url, state, start_block=start_block, end_block=end_block)

Restored previous scan state, data until block 12,619,621, we are skipping 250,000 blocks out of 250,000 total
Scanning block range 12,619,621 - 12,619,621


0it [00:00, ?it/s]

Wrote 0 PoolCreated events
Wrote 0 Swap events
Wrote 0 Mint events
Wrote 0 Burn events


### Analysing Uniswap v3 price formation

In Uniswap V3, you can get the current price of any pool from any given moment using swap events.

In [315]:
swap_df = pd.read_csv("/tmp/uniswap-v3-swap.csv")

print(f"We have total {len(swap_df):,} swaps in the dataset")

We have total 1,229,369 swaps in the dataset


Choose a pool to analyse, for example: USDC/ETH 0.3%.
You can find pool addresses [on Uniswap v3 info site](https://info.uniswap.org/#/).

In [316]:
from eth_defi.uniswap_v3.pool import fetch_pool_details

pool_address = "0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8"
pool_details = fetch_pool_details(web3, pool_address)

print(pool_details)

AttributeError: 'PoolDetails' object has no attribute 'pool_address'

Extract the swaps of this pool from the Uniswap v3 full dataset of all pools.

In [ ]:
df = swap_df.loc[swap_df.pool_contract_address == pool_address.lower()]
df

In Uniswap V3, we can get pool's price from pool's tick.

In [ ]:
def convert_price(row):
    # USDC/WETH pool has reverse token order, so let's flip it WETH/USDC
    tick = row["tick"]
    return pool_details.convert_price_to_human(tick, reverse_token_order=True)

df = df.copy(deep=True)  # https://stackoverflow.com/a/60885847/315168
df["price"] = df.apply(convert_price, axis=1)
df

Then we can convert linear price data to [OHLC candles](https://tradingstrategy.ai/docs/glossary.html#term-OHLCV).

In [ ]:
from eth_defi.research.candle import convert_to_ohlc_candles

candles = convert_to_ohlc_candles(df, time_bucket=pd.Timedelta("1D"))
candles

In [ ]:
candles.index

Now we can plot the [OHLC chart using Plotly](https://plotly.com/python/ohlc-charts/).

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Ohlc(x=candles.index,
                    open=candles['open'],
                    high=candles['high'],
                    low=candles['low'],
                    close=candles['close']))
fig.show()